## Step 1: Install Required Libraries



In [3]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

True

In [6]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
import pickle

In [7]:
real_news=pd.read_csv('/content/drive/MyDrive/Projects/True.csv')
fake_news=pd.read_csv('/content/drive/MyDrive/Projects/Fake.csv')

In [8]:
real_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [9]:
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
real_news.shape

(21417, 4)

In [11]:
fake_news.shape

(23481, 4)

In [12]:
real_news['label']=0
fake_news['label']=1

In [15]:
news=pd.concat([real_news,fake_news],axis=0).sample(frac=1).reset_index(drop=True)

In [16]:
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [17]:

news.head()

,title,text,subject,date,label
0,WATCH: George Clooney Is A Complete Idiot,"One month before the election, political activ...",left-news,"Jul 7, 2017",1
1,BREAKING VIDEO: TRUMP MOTORCADE BLASTS THROUGH...,Are these protesters at the Trump rally in Fre...,politics,"May 27, 2016",1
2,Former Iraqi president Talabani buried in Kurd...,"ERBIL, Iraq (Reuters) - Jalal Talabani, whose ...",worldnews,"October 6, 2017",0
3,Progressives Outraged Over Beyonce “So White” ...,American liberal activists still struggling wi...,Middle-east,"July 23, 2017",1
4,WOW! This Video Might Explain Why #UnfitHillar...,Notice at the beginning of the video where Joe...,left-news,"Aug 20, 2016",1


In [18]:
news.drop(['subject','date'],axis=1,inplace=True)

In [19]:
news.head()

,title,text,label
0,WATCH: George Clooney Is A Complete Idiot,"One month before the election, political activ...",1
1,BREAKING VIDEO: TRUMP MOTORCADE BLASTS THROUGH...,Are these protesters at the Trump rally in Fre...,1
2,Former Iraqi president Talabani buried in Kurd...,"ERBIL, Iraq (Reuters) - Jalal Talabani, whose ...",0
3,Progressives Outraged Over Beyonce “So White” ...,American liberal activists still struggling wi...,1
4,WOW! This Video Might Explain Why #UnfitHillar...,Notice at the beginning of the video where Joe...,1


Step2 : Cleaning of Dataset

In [21]:
import re

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove line breaks
    text = re.sub(r"\n", " ", text)

    # Remove URLs
    text = re.sub(r"https?://\S+|www\.\S+", "", text)

    # Remove HTML tags
    text = re.sub(r"<.*?>", "", text)

    # Remove punctuation
    text = re.sub(r"[^\w\s]", "", text)

    # Remove multiple newlines or extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)

    return text



Step 3: Function on columns


In [22]:
news['text']=news['text'].apply(clean_text)

Step3: Pre Processing techniques

In [23]:
stop_words=set(stopwords.words('english'))
stemming=PorterStemmer()
lemmitization=WordNetLemmatizer()

In [24]:
def preprocess(text):
  words=word_tokenize(text)
  words=[lemmitization.lemmatize(stemming.stem(word )) for word in words if word not in stop_words]
  return "".join(words)

Step 4: Applying the Function


In [25]:
news['text']=news['text'].apply(preprocess)

Step5: Feature Extraction

In [26]:
Vector=TfidfVectorizer(max_features=5000, stop_words='english')
X_tfid=Vector.fit_transform(news['text']).toarray()
y_tfid=news['label']

Step6 :Train Test Split

In [27]:
X_train,X_test,y_train,y_test=train_test_split(X_tfid,y_tfid,test_size=0.2,random_state=42)

In [28]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (35918, 5000)
Shape of y_train: (35918,)


Step7: Applying Models

In [29]:
func=MultinomialNB()
func.fit(X_train,y_train)

MultinomialNB()

In [30]:
# Train Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

Step8: Model Evaluation

In [31]:
def evaluate_model(model, X_test, y_test, name):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {acc:.2f}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

print("\nNaïve Bayes Performance:")
evaluate_model(func, X_test, y_test, "Naïve Bayes")

print("\nRandom Forest Performance:")
evaluate_model(rf_model, X_test, y_test, "Random Forest")

# Save models
with open("naive_bayes.pkl", "wb") as nb_file:
    pickle.dump(func, nb_file)
with open("random_forest.pkl", "wb") as rf_file:
    pickle.dump(rf_model, rf_file)

print("Models saved successfully!")


Naïve Bayes Performance:
Naïve Bayes Accuracy: 0.35
              precision    recall  f1-score   support

           0       0.08      0.04      0.05      4266
           1       0.42      0.63      0.50      4714

    accuracy                           0.35      8980
   macro avg       0.25      0.33      0.28      8980
weighted avg       0.26      0.35      0.29      8980

[[ 154 4112]
 [1754 2960]]

Random Forest Performance:
Random Forest Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.58      1.00      0.73      4266
           1       1.00      0.34      0.51      4714

    accuracy                           0.65      8980
   macro avg       0.79      0.67      0.62      8980
weighted avg       0.80      0.65      0.61      8980

[[4266    0]
 [3113 1601]]
Models saved successfully!
